# FastAPI Security – 10 Examples with Detailed Explanations


This notebook presents 10 FastAPI examples covering multiple security levels with proper code and explanations.

### Security Topics Covered:
1. Public Route (No Auth)  
2. HTTP Basic Authentication  
3. OAuth2 Password Flow  
4. JWT Auth (Login + Protected Route)  
5. Role-Based Access (Admin/User)  
6. API Key in Header  
7. API Key in Query Param  
8. CORS Settings  
9. HTTPS Enforcement Tips  
10. Rate Limiting using `slowapi`


## 1. Public Route (No Authentication Required)

In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/public")
def public_endpoint():
    return {"message": "This is a public API"}


✅ Anyone can access this route. No authentication required.

## 2. HTTP Basic Authentication

In [ ]:
from fastapi import Depends
from fastapi.security import HTTPBasic, HTTPBasicCredentials
import secrets

security = HTTPBasic()

@app.get("/basic-auth")
def read_current_user(credentials: HTTPBasicCredentials = Depends(security)):
    correct_username = secrets.compare_digest(credentials.username, "admin")
    correct_password = secrets.compare_digest(credentials.password, "admin123")
    if not (correct_username and correct_password):
        raise HTTPException(status_code=401, detail="Unauthorized")
    return {"username": credentials.username}


## 3. OAuth2 Password Flow (Insecure Demo Only)

In [ ]:
from fastapi.security import OAuth2PasswordBearer

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

@app.get("/oauth-protected")
def oauth_route(token: str = Depends(oauth2_scheme)):
    return {"token": token}


## 4. JWT Authentication (Login + Verify Token)

In [ ]:
import jwt
from datetime import datetime, timedelta
from fastapi import HTTPException, status

SECRET_KEY = "mysecret"
ALGORITHM = "HS256"

def create_jwt_token(data: dict):
    to_encode = data.copy()
    to_encode.update({"exp": datetime.utcnow() + timedelta(minutes=30)})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

@app.post("/login")
def login(credentials: HTTPBasicCredentials = Depends(security)):
    if credentials.username == "admin" and credentials.password == "admin123":
        token = create_jwt_token({"sub": credentials.username})
        return {"access_token": token}
    raise HTTPException(status_code=401, detail="Invalid credentials")

@app.get("/protected-jwt")
def protected_jwt(token: str = Depends(oauth2_scheme)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return {"user": payload.get("sub")}
    except:
        raise HTTPException(status_code=403, detail="Token is invalid or expired")


## 5. Role-Based Access Control (RBAC)

In [ ]:
roles_db = {
    "admin": "admin",
    "john": "user"
}

@app.get("/admin-only")
def admin_route(token: str = Depends(oauth2_scheme)):
    payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
    user = payload.get("sub")
    if roles_db.get(user) != "admin":
        raise HTTPException(status_code=403, detail="Admins only!")
    return {"admin_access_granted": user}


## 6. API Key Authentication (Header Based)

In [ ]:
from fastapi import Header

API_KEY = "my-api-key"

@app.get("/api-header")
def check_key(x_api_key: str = Header(...)):
    if x_api_key != API_KEY:
        raise HTTPException(status_code=403, detail="Invalid API Key")
    return {"access": "granted"}


## 7. API Key in Query Param

In [ ]:
@app.get("/api-query")
def check_key(api_key: str):
    if api_key != API_KEY:
        raise HTTPException(status_code=403, detail="Invalid API Key")
    return {"access": "granted via query"}


## 8. Enable CORS

In [ ]:
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Adjust for production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


## 9. Enforce HTTPS (Best Practices Only)


- Always deploy behind a reverse proxy (like Nginx) with HTTPS  
- Set `Secure` cookies  
- Redirect HTTP → HTTPS on deployment  
- Use HSTS headers via middleware


## 10. Rate Limiting with `slowapi`

In [ ]:
# pip install slowapi

from slowapi import Limiter
from slowapi.util import get_remote_address
from fastapi import Request
from slowapi.errors import RateLimitExceeded
from fastapi.responses import JSONResponse

limiter = Limiter(key_func=get_remote_address)
app.state.limiter = limiter

@app.exception_handler(RateLimitExceeded)
def rate_limit_handler(request: Request, exc: RateLimitExceeded):
    return JSONResponse(status_code=429, content={"detail": "Rate limit exceeded"})

@app.get("/limited")
@limiter.limit("3/minute")
def limited_route():
    return {"message": "You are within the limit"}
